In [2]:
from __future__ import print_function, division
import numpy as np 
import pandas as pd 
import scipy
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from skimage import transform
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import sys
import os
from imageio import imread
# print(os.listdir("../input"))

/data/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
path = glob('cityscapes/train/*')
tri = np.random.choice(path, size=1)
print(tri[0])
img = imread(tri[0])
h, w, _ = img.shape
# img.shape
# _w
_w = int(w/2)

img_A, img_B = img[:, :_w, :], img[:, _w:, :]
# img
arr = np.array([[1,2,3,4],[5,6,7,8]])
# img_A.shape
# print(arr[:1,:])
# print(arr[:])
# arr.shape
# arr = transform.resize(arr, (1,2))
# arr
import cv2
image = cv2.imread(tri[0])
image = np.fliplr(image)
# plt.imshow(image)
imgs = []
imgs.append(img_A)
imgs.append(img_B)
imgs = np.array(imgs)
imgs = np.array(imgs)/127.5 - 1.
imgs.shape
# np.fliplr

cityscapes/train/1258.jpg


(2, 256, 256, 3)

In [4]:
import cv2
resolution = (128,128)
def load_images(dataset,batch_size):
    chance = np.random.random()
    path = glob('cityscapes/train/*')
    images = np.random.choice(path, size=batch_size)
    img_real = []
    img_labelled = []
    
    for imagepath in images:
        img = cv2.imread(imagepath)
        width = img.shape[1]
        width = width//2
        real_img, labelled_img = img[:, width:, :],img[:, :width, :]
        real_img,labelled_img = transform.resize(real_img, resolution),transform.resize(labelled_img, resolution)
        if (chance<0.5):
            real_img = np.fliplr(real_img)
            labelled_img = np.fliplr(labelled_img)
        img_real.append(real_img)
        img_labelled.append(labelled_img)
    return np.array(img_labelled),np.array(img_real)

def train_load_images(dataset,batch_size):
    chance = np.random.random()
    path = glob('cityscapes/train/*')
    images = np.random.choice(path, size=batch_size)
    img_real = []
    img_labelled = []
    
    for imagepath in images:
        img = cv2.imread(imagepath)
        width = img.shape[1]
        width = width//2
        real_img, labelled_img = img[:, width:, :],img[:, :width, :]
        real_img,labelled_img = transform.resize(real_img, resolution),transform.resize(labelled_img, resolution)
        if (chance<0.5):
            real_img = np.fliplr(real_img)
            labelled_img = np.fliplr(labelled_img)
        img_real.append(real_img)
        img_labelled.append(labelled_img)
    return np.array(img_labelled),np.array(img_real)

In [12]:
# Input shape
img_rows = 128
img_cols = 128
channels = 3
img_shape = (img_rows, img_cols, channels)


# Calculate output shape of D (PatchGAN)
patch = int(img_rows / 2**4)
disc_patch = (patch, patch, 1)

# Number of filters in the first layer of G and D
filters_gen = 64
optimizer = Adam(0.0002, 0.5)

def new_generator():
    layer0 = Input(shape=img_shape,name="input")
    layer1 = Conv2D(filters_gen, kernel_size=4,strides = 2,activation = LeakyReLU(alpha=0.2),padding='same')(layer0)
    layer2 = Conv2D(filters_gen*2, kernel_size=4,strides = 2,activation = LeakyReLU(alpha=0.2),padding='same')(layer1)
    layer3 = Conv2D(filters_gen*4, kernel_size=4,strides = 2,activation = LeakyReLU(alpha=0.2),padding='same')(layer2)
    layer4 = Conv2D(filters_gen*8, kernel_size=4,strides = 2,activation = LeakyReLU(alpha=0.2),padding='same')(layer3)
    layer5 = Conv2D(filters_gen*8, kernel_size=4,strides = 2,activation = LeakyReLU(alpha=0.2),padding='same')(layer4)
    layer6 = Conv2D(filters_gen*8, kernel_size=4,strides = 2,activation = LeakyReLU(alpha=0.2),padding='same')(layer5)
    layer7 = Conv2D(filters_gen*8, kernel_size=2,strides = 2,activation = LeakyReLU(alpha=0.2),padding='same')(layer6)
    
    def deconv2d(prev_layer, skip_input, filters):
        temp = UpSampling2D(size=2)(prev_layer)
        temp = Conv2D(filters, kernel_size=4, strides=1, padding='same', activation='relu')(temp)
        temp = BatchNormalization(momentum=0.6)(temp)
        temp = Concatenate()([temp, skip_input])
        return temp
    
    u_layer1 = deconv2d(layer7, layer6, filters_gen*8)
    u_layer2 = deconv2d(u_layer1, layer5, filters_gen*8)
    u_layer3 = deconv2d(u_layer2, layer4, filters_gen*8)
    u_layer4 = deconv2d(u_layer3, layer3, filters_gen*4)
    u_layer5 = deconv2d(u_layer4, layer2, filters_gen*2)
    u_layer6 = deconv2d(u_layer5, layer1, filters_gen)
    u_layer7 = UpSampling2D(size=2)(u_layer6)
    u_layer0 = Conv2D(channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u_layer7)
    return Model(layer0,u_layer0,name="Generator")
# hell = new_generator()
# hell.summary()

def new_discriminator():
    def d_layer(prev_layer, filters):
        temp = Conv2D(filters, kernel_size=4, strides=2, padding='same')(prev_layer)
        temp = LeakyReLU(alpha=0.2)(temp)
        return temp

    layer0_A = Input(shape=img_shape)
    layer0_B = Input(shape=img_shape)
    combined_input = Concatenate(axis=-1)([layer0_A, layer0_B])
    layer1 = d_layer(combined_input, filters_gen)
    layer2 = d_layer(layer1, filters_gen*2)
    layer3 = d_layer(layer2, filters_gen*4)
    layer4 = d_layer(layer3, filters_gen*8)
    layer5 = Conv2D(1, kernel_size=4, strides=1, padding='same')(layer4)
    return Model([layer0_A, layer0_B], layer5,name="Discriminator")

# heaven = new_discriminator()
# heaven.summary()


In [18]:
'''
1. Two images as input.
2. One image goes to generator
3. Two images go to the discriminator as input
4. We freeze the discriminator weights
5. Create GAN with both models. Inputs as earlier and outputs as earlier
'''
d = new_discriminator()
d.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])
#Since the model is being compiled already here, the flag later of trainable = False does not affect it.

g = new_generator()

A = Input(shape = img_shape)
B = Input(shape = img_shape)

fake_image_B = g(B)
validity = d([fake_image_B,B])

#THIS ONLY STOPS THE TRAINING OF THE DISCRIMINATOR IN THE GAN. IT WILL STILL TRAIN WHEN INVOKED AS JUST D.
d.trainable = False

gan = Model(inputs=[A,B],outputs = [validity,fake_image_B],name="hey")

gan.compile(loss=['mse', 'mae'],
                              loss_weights=[1, 100],
                              optimizer=optimizer)

d.summary()

/data/anaconda/envs/py35/lib/python3.5/site-packages/keras/activations.py:211: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_45 (InputLayer)           (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
input_46 (InputLayer)           (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
concatenate_85 (Concatenate)    (None, 128, 128, 6)  0           input_45[0][0]                   
                                                                 input_46[0][0]                   
__________________________________________________________________________________________________
conv2d_229 (Conv2D)             (None, 64, 64, 64)   6208        concatenate_85[0][0]             
__________

/data/anaconda/envs/py35/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [13]:
# Input shape
img_rows = 128
img_cols = 128
channels = 3
img_shape = (img_rows, img_cols, channels)


# Calculate output shape of D (PatchGAN)
patch = int(img_rows / 2**4)
disc_patch = (patch, patch, 1)

# Number of filters in the first layer of G and D
gf = 64
df = 64

optimizer = Adam(0.0002, 0.5)

# Build and compile the discriminator
discriminator = build_discriminator()
discriminator.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

# Build the generator
generator = build_generator()

# Input images and their conditioning images
img_A = Input(shape=img_shape,name='img_A')
img_B = Input(shape=img_shape)

# By conditioning on B generate a fake version of A
fake_A = generator(img_B)

# For the combined model we will only train the generator
discriminator.trainable = False

# Discriminators determines validity of translated images / condition pairs
valid = discriminator([fake_A, img_B])

combined = Model(inputs=[img_A, img_B], outputs=[valid, fake_A])
combined.compile(loss=['mse', 'mae'],
                              loss_weights=[1, 100],
                              optimizer=optimizer)

valid = np.ones((batch_size,) + disc_patch)
print(valid)

NameError: name 'build_discriminator' is not defined

In [16]:
def show_images( dataset_name,epoch, batch_i):
        
        r, c = 3, 3

        imgs_A, imgs_B = load_data(dataset_name,batch_size=3, is_val=True)
        fake_A = generator.predict(imgs_B)

        gen_imgs = np.concatenate([imgs_B, fake_A, imgs_A])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Input', 'Output', 'Ground Truth']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[i])
                axs[i,j].axis('off')
                cnt += 1
        plt.show()
        plt.close()
        
def train( dataset_name,epochs, batch_size=1, show_interval=10):

        start_time = datetime.datetime.now()

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + disc_patch)
        fake = np.zeros((batch_size,) + disc_patch)

        for epoch in range(epochs):
            for batch_i, (imgs_A, imgs_B) in enumerate(load_batch(dataset_name,batch_size)):

                
                #  Train Discriminator
                

                # Condition on B and generate a translated version
                fake_A = generator.predict(imgs_B)

                # Train the discriminators (original images = real / generated = Fake)
                d_loss_real = discriminator.train_on_batch([imgs_A, imgs_B], valid)
                d_loss_fake = discriminator.train_on_batch([fake_A, imgs_B], fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

               
                #  Train GAN
                g_loss = combined.train_on_batch([imgs_A, imgs_B], [valid, imgs_A])

                elapsed_time = datetime.datetime.now() - start_time
                
            # Plot the progress
            if epoch%10==0:
                  print ("[Epoch %d/%d]  [D loss: %f, acc: %3d%%] [G loss: %f] time: %s" % (epoch, epochs,
                                                                        
                                                                        d_loss[0], 100*d_loss[1],
                                                                        g_loss[0],
                                                                        elapsed_time))
            # If at show interval => show generated image samples
            if epoch % show_interval == 0:
                    show_images(dataset_name,epoch, batch_i)

In [ ]:
train("cityscapes",epochs=50, batch_size=32, show_interval=10)
# print(os.listdir("cityscapes"))
# !mkdir input
# patch = int(img_rows / 2**4)
# disc_patch = (patch, patch, 1)
# disc_patch